In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import skimage as sk
from skimage import measure
import os
import tifffile
from tqdm import tqdm

In [2]:
dots_data = pd.read_csv("field_001.gated_dots.tsv", sep="\t")

In [3]:
dots_data2 = dots_data.loc["20x" == dots_data["magnification"], :]

In [4]:
dots_data2

,series_id,scaling,magnification,image_type,Channel,Nuclei,x,y,z,Value,FWHM,SNR,NSNR,Label,SNR2,nid,Value2,selected
2781,1,0.25575,20x,dw,a647,11,320.953,49.991,55.139,64.684,1.225,131.287,43.313,0,44.147016,84,252.918866,True
2782,1,0.25575,20x,dw,a647,38,414.198,124.273,54.872,44.279,1.321,95.345,33.723,0,61.835469,105,173.133920,True
2783,1,0.25575,20x,dw,a647,0,80.611,256.977,54.940,43.324,1.364,22.159,-1.000,0,13.620983,7,169.399804,True
2784,1,0.25575,20x,dw,a647,0,517.900,383.024,49.818,40.886,1.418,192.486,-1.000,0,135.620985,121,159.867058,True
2785,1,0.25575,20x,dw,a647,8,403.852,19.771,56.064,37.710,1.491,4.220,31.201,0,4.722772,94,147.448680,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4285,1,0.25575,20x,dw,a647,131,509.808,546.042,51.929,2.388,2.019,11.079,2.639,0,8.053350,130,9.337243,True
4286,1,0.25575,20x,dw,a647,0,345.976,406.952,54.898,2.385,1.590,14.840,-1.000,0,9.652582,99,9.325513,True
4287,1,0.25575,20x,dw,a647,52,356.035,168.908,47.034,2.383,1.332,9.653,2.248,0,8.608129,98,9.317693,True
4288,1,0.25575,20x,dw,a647,124,442.171,602.917,47.135,2.310,1.718,11.625,4.141,0,10.628320,111,9.032258,True


In [5]:
ref__dw = dots_data2.loc["dw" == dots_data2["image_type"], :].reset_index(drop=True)
dw__image_folder_path = "/mnt/data/Imaging/202105-Deconwolf/data_210726/20x_dw"
mask_folder_path = "../../data/20x_mask/dilated_labels_watershed_from60x"

In [6]:
current_field_id = 1

print(f"Field #{current_field_id}")

dw__max_z_proj = tifffile.imread(os.path.join(dw__image_folder_path, f"a647_{current_field_id:03d}.tif")).max(0)

labels = tifffile.imread(os.path.join(mask_folder_path, f"a647_{current_field_id:03d}.dilated_labels.from_60x.tiff")).reshape(dw__max_z_proj.shape)

field_dw__dots = ref__dw.loc[ref__dw["series_id"] == current_field_id, :].sort_values("Value2", ascending=False)

selected_dw__dots = field_dw__dots.reset_index(drop=True)

fig3, ax = plt.subplots(figsize=(10, 10), ncols=1, constrained_layout=True)
fig3.suptitle(f"Field #{current_field_id}")
print(" > Plotting dw")
ax.set_title(f"60x_dw (n.dots={selected_dw__dots.shape[0]})")
ax.imshow(
    dw__max_z_proj, cmap=plt.get_cmap("gray"), interpolation="none",
    vmin=dw__max_z_proj.min()*1.5, vmax=dw__max_z_proj.max()*.5,
    resample=False, filternorm=False)
ax.scatter(
    x=selected_dw__dots["y"].values,
    y=selected_dw__dots["x"].values,
    s=30, facecolors='none', edgecolors='r', linewidth=.5)
print(" > Plotting contours")
for lid in tqdm(range(1, labels.max()), desc="nucleus"):
    contours = measure.find_contours(labels == lid, 0.8)
    for contour in contours:
        ax.scatter(x=contour[:,1], y=contour[:,0], c="yellow", s=.005)
plt.close(fig3)
print(" > Exporting")
fig3.savefig(f"overlay_{current_field_id:03d}.20x.png", bbox_inches='tight')
print(" ! DONE")

nucleus:   0%|          | 0/180 [00:00<?, ?it/s]

Field #1
 > Plotting dw
 > Plotting contours


nucleus: 100%|██████████| 180/180 [00:02<00:00, 78.43it/s]


 > Exporting
 ! DONE
